<ul>
<li>Provide a list of audiences activated each quarter, broken down by vehicle.</li>
<li>Include details about the activation path, such as direct vs. NS vs. LR, and the use of Permutive.</li>
<li>Describe the nature of the audience, e.g., whether it's 3p (third-party) or 1p (first-party), etc.</li>
<li>Additionally, please specify the associated media spend for each audience.</li>
<li>Include information about the funnel stage, including segments like high propensity, active, etc.</li>
<li>Present performance results, with a preference for comparing changes to baseline data if it is available.</li>
<ul>

This analysis uses the output of the "Delve File Manipulation"

<h2>Library</h2>

In [70]:
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

<h2>Function</h2>

<h2>Main</h2>

In [71]:
folder_path = r'C:\Users\Rafael_Fagundes\Downloads\compiled_data.csv'

df = pd.read_csv(folder_path)

In [72]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1711983 entries, 0 to 1711982
Data columns (total 48 columns):
 #   Column                                       Dtype  
---  ------                                       -----  
 0   Unnamed: 0                                   int64  
 1   Audience Type                                object 
 2   Tactic                                       object 
 3   Media Buy Name                               object 
 4   Campaign Name                                object 
 5   Fiscal Quarter                               object 
 6   Fiscal Week                                  object 
 7   Country_x                                    object 
 8   Display Funnel Mapped                        object 
 9   Display Dell Vehicle Mapped                  object 
 10  Display Business Unit Mapped                 object 
 11  Display Site Name Mapped                     object 
 12  Audience Type Name                           object 
 13  Impressions 

In [73]:
df = df[(df['Segment ID'] != 0) & (df['Segment ID'] != '0')]


In [74]:
df_audiences = df[['Country_x', 'Fiscal Quarter', 'BU', 'Segment ID', 'Spend', 'Net Rev']]

df_audiences = df_audiences.groupby(['Segment ID', 'Country_x', 'BU', 'Fiscal Quarter'])[['Spend', 'Net Rev']].sum().reset_index()


In [75]:
df_audiences.head()

,Segment ID,Country_x,BU,Fiscal Quarter,Spend,Net Rev
0,d10upqm9p51,United States,CSB,2024-Q1,0.000000,0.000000
1,d10upqm9p51,United States,CSB,2024-Q2,43012.465726,5604.530000
2,d10upqm9p51,United States,CSB,2024-Q3,1270.339814,47.990000
3,d14s5u5pfra,France,CSB,2024-Q3,1579.502062,4495.353822
4,d17vaylat7k,Canada,CSB,2024-Q2,0.000000,0.000000


In [76]:
# Create a new column 'Status' to initially mark all segments as 'New'
df_audiences['Status'] = 'New'

# Iterate through each unique 'Segment ID'
for audience_id in df_audiences['Segment ID'].unique():
    # Find the rows where 'Segment ID' matches the current ID
    segment_rows = df_audiences[df_audiences['Segment ID'] == audience_id]
    
    # Check if there is more than one row for this segment (i.e., it was reused)
    if len(segment_rows) > 1:
        # Set 'Status' to 'Reused' for all rows except the first one
        df_audiences.loc[segment_rows.index[1:], 'Status'] = 'Reused'

In [77]:
pivot_df = df_audiences.pivot_table(
    index=['Country_x', 'BU', 'Status'],
    columns='Fiscal Quarter',
    values='Segment ID',
    aggfunc='count',
    margins=True,  # Add a total line
    margins_name='Total'  # Name for the total line
)

# Format the pivot table cells as integers and fill NaN values with an empty string
pivot_df = pivot_df.applymap(lambda x: f'{int(x):,}' if not pd.isna(x) else '')

pivot_df


Fiscal Quarter           2023-Q4 2024-Q1 2024-Q2 2024-Q3 Total
Country_x     BU  Status                                      
Australia     CSB New                                  3     3
Canada        B2B New                          7             7
                  Reused                               4     4
              CSB New                 24      11       4    39
                  Reused                      24      19    43
France        CSB New                                  1     1
United States B2B New                 33      26            59
                  Reused                      33      59    92
              CSB New          3      45       9       1    58
                  Reused               3      47      48    98
Total                          3     105     157     139   404